In [ ]:
# Include shared notebooks
%run "../includes/configuration.ipynb"
%run "../includes/common_functions.ipynb"

In [ ]:
# rename to differentiate columns from population file
expnd_df = spark.read.format("parquet").load(f"{processed_folder_path}/expenditure") \
.withColumnRenamed("eofy", "end_of_fy") \
.withColumnRenamed("jurisdiction", "expnd_jurisdiction")

In [ ]:
# aggregate by year, state, and source_of_funds
from pyspark.sql.functions import sum

fund_source_df = expnd_df \
    .groupBy("year", "expnd_jurisdiction", "source_of_funds", "end_of_fy") \
    .agg(sum("current_amount").alias("total_amount"))

In [ ]:
# load population file with selected columns
pop_df = spark.read.format("parquet").load(f"{processed_folder_path}/population") \
.withColumnRenamed("jurisdiction", "pop_jurisdiction")

In [ ]:
# join dfs
join_condition = [ fund_source_df.end_of_fy == pop_df.Time, fund_source_df.expnd_jurisdiction == pop_df.pop_jurisdiction ]
fund_source_with_pop_df = fund_source_df.join(pop_df, join_condition)

In [ ]:
# add calculated column amount_per_person, and created_date column
from pyspark.sql.functions import current_timestamp, expr

final_df = fund_source_with_pop_df \
                .withColumn("amount_per_person", expr("total_amount / population")) \
                .withColumn("created_date", current_timestamp())

In [ ]:
#write the result file to presentation layer
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/fund_source_with_pop")